In [1]:
import os
import zipfile
import time
import pickle
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import load_pickle, dump_pickle, get_feature_value, feature_spearmanr, feature_target_spearmanr, addCrossFeature, calibration
from utils import raw_data_path, feature_data_path, cache_pkl_path, analyse

from sklearn.metrics import log_loss
import lightgbm as lgb

In [2]:
all_data_path = feature_data_path + 'all_data_all_features_new_0512.pkl'
all_data = load_pickle(all_data_path)

In [3]:
target = 'is_trade'

features = load_pickle('all_features_day_4567.pkl')
categorical_feature = load_pickle('categorical_feature.pkl')

len(features), len(categorical_feature)

(256, 4)

In [4]:
data = all_data[all_data.is_trade >= 0]
all_data = None
gc.collect()

train_data = data[(data.hour < 10) | (data.day < 7)]
test_data_1 = data[(data.hour == 10) & (data.day == 7)]
test_data_2 = data[(data.hour == 11) & (data.day == 7)]

data = None
gc.collect()

lgb_train_data = lgb.Dataset(train_data[features], label=train_data[target], feature_name=features, categorical_feature=categorical_feature)
lgb_test_data_1 = lgb_train_data.create_valid(test_data_1[features], label=test_data_1[target])
lgb_test_data_2 = lgb_train_data.create_valid(test_data_2[features], label=test_data_2[target])

train_data = None
gc.collect()

test_data_1.shape, test_data_2.shape

((142795, 573), (126942, 573))

In [5]:
lgb_train_data.save_binary(raw_data_path + 'lgb4567_until_10_train')
lgb_test_data_1.save_binary(raw_data_path + 'lgb4567_hour_10_train')
lgb_test_data_2.save_binary(raw_data_path + 'lgb4567_hour_11_train')

In [ ]:
lgb_train_data = lgb.Dataset(raw_data_path + 'lgb4567_until_10_train')
lgb_test_data_1 = lgb.Dataset(raw_data_path + 'lgb4567_hour_10_train')
lgb_test_data_2 = lgb.Dataset(raw_data_path + 'lgb4567_hour_11_train')

lgb_train_data.set_feature_name(features)
lgb_train_data.set_categorical_feature(categorical_feature)